In [5]:
import pandas as pd
import numpy as np

import plotly 
# import plotly.plotly as py
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
py.offline.init_notebook_mode(connected=True)
from fbprophet.plot import plot_plotly
import plotly.offline as py
import plotly.offline as ofl
import plotly.tools as tls

#mute unimportant diagnostic messages
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import os
os.getcwd()

'/Users/nikolasdawson/Documents/University/UTS/Journalism_Jobs/AUS/data'

## ABS Labour Force Data
### Labour Force, Australia, Detailed, Quarterly, Nov 2019
Unit Level: EQ08 - Employed persons by Occupation unit group of main job (ANZSCO), Sex, State and Territory, August 1986 onwards
https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/6291.0.55.003Nov%202019?OpenDocument

In [2]:
unit_quarterly = pd.read_csv('ANZSCOUnit_Quarterly.csv', header=0, sep=',', 
                             parse_dates=['Mid-quarter month'], dayfirst=True)
unit_quarterly

,Mid-quarter month,Sex,State,ANZSCOUnitCode,ANZSCOUnitName,ANZSCOCombined,Employed total ('000),Number of hours actually worked in all jobs ('000 Hours)
0,1986-08-01,Males,New South Wales,1111,Chief Executives and Managing Directors,1111 Chief Executives and Managing Directors,10.373967,520.688181
1,1986-08-01,Males,New South Wales,1112,General Managers,1112 General Managers,9.112449,457.370343
2,1986-08-01,Males,New South Wales,1113,Legislators,1113 Legislators,0.856575,53.397057
3,1986-08-01,Males,New South Wales,1211,Aquaculture Farmers,1211 Aquaculture Farmers,0.349588,18.166174
4,1986-08-01,Males,New South Wales,1212,Crop Farmers,1212 Crop Farmers,9.303512,483.214122
...,...,...,...,...,...,...,...,...
622986,2019-11-01,Females,Australian Capital Territory,8113,Domestic Cleaners,8113 Domestic Cleaners,0.499780,8.032732
622987,2019-11-01,Females,Australian Capital Territory,8116,Other Cleaners,8116 Other Cleaners,0.343426,14.411873
622988,2019-11-01,Females,Australian Capital Territory,8513,Kitchenhands,8513 Kitchenhands,0.858586,22.624640
622989,2019-11-01,Females,Australian Capital Territory,8912,Shelf Fillers,8912 Shelf Fillers,0.823155,14.462972


#### Filter for `Journalists and Other Writers` unit group, ANZSCOUnitCode = `2124`

In [4]:
journalists = unit_quarterly[unit_quarterly['ANZSCOUnitCode'] == 2124]

In [6]:
journalists.columns

Index(['Mid-quarter month', 'Sex', 'State', 'ANZSCOUnitCode', 'ANZSCOUnitName',
       'ANZSCOCombined', 'Employed total ('000)',
       'Number of hours actually worked in all jobs ('000 Hours)'],
      dtype='object')

#### Group by `Mid-quarter month`, summing the `Sex` and `State` rows so we get national values

In [15]:
journalists_grouped = journalists.groupby(['Mid-quarter month']).sum()
journalists_grouped = journalists_grouped.reset_index()
#Drop the Unit Code now, we don't need it
journalists_grouped = journalists_grouped.drop(columns=['ANZSCOUnitCode'])
journalists_grouped

,Mid-quarter month,Employed total ('000),Number of hours actually worked in all jobs ('000 Hours)
0,1986-08-01,12.794435,504.504982
1,1986-11-01,14.231606,568.299381
2,1987-02-01,17.252471,666.641887
3,1987-05-01,14.664992,541.174778
4,1987-08-01,13.668324,525.456883
...,...,...,...
129,2018-11-01,23.369009,787.637604
130,2019-02-01,17.668741,528.341211
131,2019-05-01,19.709453,548.273350
132,2019-08-01,26.997818,832.728032


In [16]:
# We only want to keep 2012 and beyond
journalists_grouped = journalists_grouped[journalists_grouped['Mid-quarter month'] >= '2012-01-01']
journalists_grouped

,Mid-quarter month,Employed total ('000),Number of hours actually worked in all jobs ('000 Hours)
102,2012-02-01,26.184923,926.638355
103,2012-05-01,31.768270,1168.440190
104,2012-08-01,32.453228,1175.121105
105,2012-11-01,27.622757,941.773591
106,2013-02-01,28.515415,942.216798
107,2013-05-01,15.986028,531.356315
108,2013-08-01,23.487987,807.733921
109,2013-11-01,21.199998,747.433897
110,2014-02-01,23.467102,807.547282
111,2014-05-01,26.634632,1010.085207


## Visualise data

In [22]:
x = journalists_grouped['Mid-quarter month']
y0 = journalists_grouped["Employed total ('000)"]


# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=x, y=y0,
               name="Journalists & Other Writers (Unit Level)",
              line = dict(color='orange', width=4))
)

# Add figure title
fig.update_layout(
    yaxis_title="Journalists Employed Quarterly (000's)",
    font=dict(
        family="Arial",
        size=20,
        color="#7f7f7f"),
    margin=dict(l=20, r=20, t=80, b=80),
    legend=dict(x=0, y=1.3)
)

# Set x-axis title
# fig.update_xaxes(title_text="Quarter")

fig.update_xaxes(showgrid=True, ticks="outside", tickwidth=2, tickcolor='white', ticklen=10)
fig.update_yaxes(zeroline=True, zerolinewidth=10, showgrid=True)

fig = go.FigureWidget(fig)
fig.layout.template = 'plotly_white'

fig.show()